<a href="https://colab.research.google.com/github/SpirinEgor/gulag/blob/master/notebooks/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture

from os import environ

# Set this to `True` to train on Colab
ON_CLOUD = True

if ON_CLOUD:
    from getpass import getpass
    from urllib.parse import quote

    token = getpass("GitHub token: ")
    token = quote(token)
    environ["GITHUB_TOKEN"] = token
    ! if [ -d gulag ]; then rm -Rf gulag; fi
    ! git clone https://$GITHUB_TOKEN@github.com/SpirinEgor/gulag.git
    %cd gulag
    ! pip install -q -r requirements.txt
else:
    %cd ..

In [ ]:
import gin

from src.data import MultiLanguageClassificationDataModule
from src.main import train
from src.utils import setup_logging

setup_logging()

In [ ]:
gin.parse_config_file("config/train.gin")
gin.bind_parameter("train.n_steps", 20_000)

print(gin.config_str())

In [ ]:
gin.parse_config_file("config/train.gin")
gin.bind_parameter("train.n_steps", 20_000)

print(gin.config_str())

# Data overview

Some examples from synthetic dataset

In [ ]:
data_module = MultiLanguageClassificationDataModule(batch_size=1)
data_module.setup()

In [ ]:
for i, batch in enumerate(data_module.train_dataloader()):
    if i == 5:
        break
    seq, attn, labels = batch

    orig_str = data_module.tokenizer.decode(seq[0])
    class_names = data_module.decode_languages(labels[0])

    print(
        f"""{'=' * 20}
Input tokens (len = {seq.shape[-1]}):
{seq[0]}
Target classes:
{class_names}
Original string:
{orig_str}
{'=' * 20}
"""
    )

In [7]:
for i, batch in enumerate(data_module.train_dataloader()):
    if i == 5:
        break
    seq, attn, labels = batch

    orig_str = data_module.tokenizer.decode(seq[0])
    class_names = data_module.decode_languages(labels[0])

    print(
        f"""{'=' * 20}
Input tokens (len = {seq.shape[-1]}):
{seq[0]}
Target classes:
{class_names}
Original string:
{orig_str}
{'=' * 20}
"""
    )

Input tokens (len = 88):
tensor([   101,  11641,  17513,  16376,  27830,  11130,  37398,  29343,  64246,
         10263,  81506,  22623,  19048,  10357,  10134,  16478,  10155,  10105,
         26574,  22584,  12602,  74935,  10115,  16917,  32973,  12904,  10155,
         10105,  11572,  10108,  10105,  10924,    743,  14341,    732,  40361,
         18476,    746,  57394,  86147,  10416,    727,  12534,  69098, 100518,
         41263,    727,  48311,  53631,  12726,    726,  17923,  52220,    737,
         33915,  86983,  11828,  37009,  93066,  20023,    727,  70112,  17085,
         16289,  26688,  12514,  18594,  38476,  10416,  79468,  65546,  10507,
         49929,  13834,  15463,    727,  25802,  66362, 101989, 107835,  22260,
         16302,  30372,  10886,  16306,  67339,  86210,    102])
Target classes:
['[NOT LANG]', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en

# Train model

In [9]:
train()

Global seed set to 7
wandb: Currently logged in as: voudy (use `wandb login --relogin` to force relogin)


INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /tmp/tmprz4a1c9t
INFO:torch.distributed.nn.jit.instantiator:Writing /tmp/tmprz4a1c9t/_remote_module_non_sriptable.py
loading configuration file https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1445: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `test(ckpt_path='best')` to use and best model checkpoint and avoid this warning or `ckpt_path=trainer.checkpoint_callback.last_model_path` to use the last model.
  f"`.{fn}(ckpt_path=None)` was called without a model."
INFO:src.data.data_module:Downloading and opening 'wikiann' dataset for ru, uk, be, kk, az, hy, ka, he, en, de
INFO:datasets.info:Loading Dataset Infos from /root/.cache/huggingface/modules/datasets_modules/datasets/wikiann/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e
INFO:datasets.builder:Overwrite dataset info from restored data version.
INFO:datasets.info:Loading Dataset info from /root/.cache/huggingface/datasets/wikiann/ru/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e
INFO:datasets.info:Loading Dataset info f

Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         test/f1            0.9504076838493347
        test/loss           0.14909984171390533
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr-AdamW,███▆▅▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test/f1,▁
test/loss,▁
train/f1,▁
train/loss,▁
train/step_f1,▁▃▄▆▆▃▄▇▆▆▆▅▅▆▆▆▆▅▅▆▆▆▇▆▂▅▇▆▆▄▆▃▆▆▆▆▅▇▅█
train/step_loss,█▆▅▃▃▅▄▂▃▂▄▃▄▃▃▃▃▃▃▃▂▃▂▃▅▃▂▂▃▄▃▅▂▃▃▂▃▂▃▁
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val/f1,▁▄▅▅▆▆▇▇▇▇▇▇█▇▇█████
val/loss,█▅▄▄▃▃▂▃▂▂▂▂▁▂▂▁▁▂▁▁
